input : N_RUNS, TASKS, ALPHAS, ACT_FUNCTION, CLASS_METRICS, connectivity(+sub_id)/rsn file, input/output node

output : dataset(run==0), 

         train/test reservoir state(run==0 & alpha==1) 

         decision function, predict/ground truth(run==0 & alpha==1) 

         특정 subj의 connectiviy를 사용한 readout model의 성능 csv file
         
              {activation, run, alpha, metric1점수, metric2점수, ...}

In [ ]:
import warnings
import os
import numpy as np
from conn2res.tasks import NeuroGymTask
from conn2res.connectivity import Conn
from conn2res.reservoir import EchoStateNetwork
from conn2res.readout import Readout
import matplotlib.pyplot as plt
from conn2res import readout, plotting

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
# ~/_conn2res
PROJ_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  
# ~/_conn2res/experiments/figs 
OUTPUT_DIR = os.path.join(PROJ_DIR, "experiments", 'figs')
if not os.path.isdir(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
# 각 task에서 각 activation func마다 실행하는 run 횟수(1 run = 1000 trials)
N_RUNS = 50 
TASKS = [
    'ContextDecisionMaking',
]
# readout model 성능 평가 기준
CLASS_METRICS = [
    'balanced_accuracy_score',
    'f1_score',
]
ALPHAS = np.linspace(0, 2, 41)[1:]
ACT_FCNS = [
    'tanh',
    'sigmoid',
    'relu',
]
# connectivity matrix heatmap 시각화 여부
VIZ_CONN = False

In [ ]:
# individual일 시, id:0~69
conn = Conn(subj_id=0, filename='consensus_human_250.npy')  
# weight를 [0,1]로 scale후 spectral radius로 normalization
conn.scale_and_normalize()  
if conn.w.ndim == 3:
    print(f'\n Connectivity Matrix : Individual, ROI수 = {conn.n_nodes}')
else:
    print(f'\n Connectivity Matrix : Consensus, ROI수 = {conn.n_nodes}')
    
# conenctivity matrix heatmap
if VIZ_CONN == True:
    w_log=np.log(conn.w)
    w_log=np.nan_to_num(w_log, neginf=-12.5)

    print(np.min(w_log), np.max(w_log))

    fig,ax1=plt.subplots()
    w_show=ax1.imshow(w_log, cmap='turbo', interpolation= 'nearest')
    cbar=ax1.figure.colorbar(w_show, ax=ax1)
    #sns.jointplot(data=w_log, kind='scatter')
    plt.show()